# Exploratory Data Analysis, Data Pre Processing and Feature selection

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.impute import SimpleImputer
from sklearn.impute import KNNImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import RobustScaler



from sklearn.feature_selection import RFE

from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LassoCV

#from sklearn.metrics import f1_score, confusion_matrix, recall_score

In [7]:
#Y
df = pd.read_csv('C:/Users/Estevao/Desktop/Nova IMS/2nd Semester/ML Operations/Final Project Structure/project/hospital-full-project/hospital-full-project/data/01_raw_data/train.csv').set_index('encounter_id')

In [8]:
#Y
X_test = pd.read_csv('C:/Users/Estevao/Desktop/Nova IMS/2nd Semester/ML Operations/Final Project Structure/project/hospital-full-project/hospital-full-project/data/01_raw_data/test.csv').set_index('encounter_id')

In [4]:
def prepare_data(df):

    train = df.drop(columns=['readmitted_binary', 'readmitted_multiclass'])
    target_binary = df['readmitted_binary']
    return train, target_binary

In [5]:
def process_data(train, X_test):
    # Step 1: Add hospital_visits column
    train['hospital_visits'] = train.groupby('patient_id')['patient_id'].transform('count')
    X_test['hospital_visits'] = X_test.groupby('patient_id')['patient_id'].transform('count')
    
    # Step 3: Calculate Emergency_visits/total_visits ratio
    train['Emergency_visits/total_visits'] = train['emergency_visits_in_previous_year'] / (
        train['inpatient_visits_in_previous_year'] + train['outpatient_visits_in_previous_year'] + train['emergency_visits_in_previous_year'])
    X_test['Emergency_visits/total_visits'] = X_test['emergency_visits_in_previous_year'] / (
        X_test['inpatient_visits_in_previous_year'] + X_test['outpatient_visits_in_previous_year'] + X_test['emergency_visits_in_previous_year'])
    
    # Step 4: Calculate n_medications/length_of_stay ratio
    train['n_medications/length_of_stay'] = train['number_of_medications'] / train['length_of_stay_in_hospital']
    X_test['n_medications/length_of_stay'] = X_test['number_of_medications'] / X_test['length_of_stay_in_hospital']
    
    return train, X_test


In [6]:
def clean_data(df):
    # Strip whitespace from string values
    df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)
    
    # Replace specific placeholder values with NaN
    df.replace('?', np.nan, inplace=True)
    df.replace('Unknown/Invalid', np.nan, inplace=True)
    df.replace(['Not Mapped', 'Not Available'], np.nan, inplace=True)
    
    # Drop the 'country' column if it exists
    if 'country' in df.columns:
        df.drop('country', axis=1, inplace=True)
    
    # Fill NaNs in specific columns with 'Not_tested'
    if 'glucose_test_result' in df.columns:
        df['glucose_test_result'].fillna('Not_tested', inplace=True)
    if 'a1c_test_result' in df.columns:
        df['a1c_test_result'].fillna('Not_tested', inplace=True)
    
    # Map 'payer_code' column: NaN to 0, others to 1
    if 'payer_code' in df.columns:
        df['payer_code'] = df['payer_code'].map(lambda x: 0 if pd.isna(x) else 1)
    
    return df

def process_age_column(df):
    def col_age(age):
        if not pd.isna(age):
            new_value = age.split('-')
            age_1 = int(new_value[0].strip('['))
            age_2 = int(new_value[1].strip(')'))
            return (age_1 + age_2) / 2
        else:
            return np.nan
    
    if 'age' in df.columns:
        df['age'] = df['age'].apply(col_age)
    
    return df

def drop_newborn_outliers(df, target_df):
    # Find indices to drop
    to_drop = df[(df['admission_type'] == 'Newborn') & (df['age'] > 5)].index.to_list()
    # Drop rows in the main DataFrame
    df.drop(to_drop, inplace=True)
    # Drop rows in the target DataFrame
    target_df.drop(to_drop, inplace=True)
    return df, target_df

def drop_newborn_outliers1(df):
    # Find indices to drop
    to_drop = df[(df['admission_type'] == 'Newborn') & (df['age'] > 5)].index.to_list()
    # Drop rows in the main DataFrame
    df.drop(to_drop, inplace=True)
    return df

def fill_missing_age(train, test):
    # Fill missing 'age' values in train set
    train['age'] = train.groupby('patient_id')['age'].transform(lambda x: x.fillna(x.mean()))
    # Calculate mean age per patient
    train_patient_means = train.groupby('patient_id')['age'].mean()
    # Fill missing 'age' values in test set using train's patient means
    test['age'] = test['patient_id'].map(train_patient_means).where(test['age'].isnull(), test['age'])
    return train, test

def fill_missing_race(train, test):
    # Fill missing 'race' values in train set
    train['race'] = train.groupby('patient_id')['race'].transform(lambda x: x.fillna(x.mode().iloc[0] if not x.mode().empty else np.nan))
    # Calculate mode race per patient
    modes = train.groupby('patient_id')['race'].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)
    # Fill missing 'race' values in test set using train's patient modes
    test['race'] = test['patient_id'].map(modes).where(test['race'].isnull(), test['race'])
    return train, test

def impute_missing_values(train, test):
    # Impute missing values for numeric columns with mean
    numeric = train.select_dtypes(include=np.number).columns
    imputer_numeric = SimpleImputer(strategy='mean')
    imputer_numeric.fit(train[numeric])
    train[numeric] = imputer_numeric.transform(train[numeric])
    test[numeric] = imputer_numeric.transform(test[numeric])
    
    # Impute missing values for categorical columns with the most frequent value
    object_columns = train.select_dtypes(include=['object']).columns
    imputer_object = SimpleImputer(strategy='most_frequent')
    imputer_object.fit(train[object_columns])
    train[object_columns] = imputer_object.transform(train[object_columns])
    test[object_columns] = imputer_object.transform(test[object_columns])
    
    return train, test

def map_icd9_to_category(df):
    def icd9_to_category(icd9):
        if not pd.isna(icd9):
            if icd9[0] == 'E' or icd9[0] == 'V':
                return 'other health factors and external causes'
            else:
                icd9 = float(icd9)
                if icd9 >= 1 and icd9 <= 139:
                    return 'infectious and parasitic diseases'
                elif icd9 >= 140 and icd9 <= 239:
                    return 'neoplasms'
                elif icd9 >= 240 and icd9 <= 279:
                    return 'diabetes, endocrine and metabolic disorders'
                elif icd9 >= 280 and icd9 <= 289:
                    return 'other diseases/conditions'
                elif icd9 >= 290 and icd9 <= 319:
                    return 'mental disorders'
                elif icd9 >= 320 and icd9 <= 389:
                    return 'other diseases/conditions'
                elif icd9 >= 390 and icd9 <= 459:
                    return 'circulatory system diseases'
                elif icd9 >= 460 and icd9 <= 519:
                    return 'respiratory system diseases'
                elif icd9 >= 520 and icd9 <= 579:
                    return 'digestive system diseases'
                elif icd9 >= 580 and icd9 <= 629:
                    return 'genitourinary system diseases'
                elif icd9 >= 630 and icd9 <= 679:
                    return 'other diseases/conditions'
                elif icd9 >= 680 and icd9 <= 709:
                    return 'skin diseases'
                elif icd9 >= 710 and icd9 <= 739:
                    return 'musculoskeletal disorders'
                elif icd9 >= 740 and icd9 <= 759:
                    return 'other diseases/conditions'
                elif icd9 >= 760 and icd9 <= 779:
                    return 'perinatal conditions'
                elif icd9 >= 780 and icd9 <= 799:
                    return 'uncertain conditions'
                elif icd9 >= 800 and icd9 <= 999:
                    return 'injury and poisoning'
                elif icd9 == 0:
                    return 'None'
        else:
            return np.nan
    
    columns_to_map = ['primary_diagnosis', 'secondary_diagnosis', 'additional_diagnosis']
    for col in columns_to_map:
        df[col] = df[col].apply(icd9_to_category)
    
    return df

def create_diagnosis_columns(train, X_test):
    # Step 1: Generate unique diagnosis values
    diagnosis_values = train['primary_diagnosis'].unique().tolist() + \
                       train['secondary_diagnosis'].unique().tolist() + \
                       train['additional_diagnosis'].unique().tolist()
    diagnosis_values = list(set(diagnosis_values))
    
    # Step 2: Create binary columns for each unique diagnosis value in train
    for i in diagnosis_values:
        train['diagnosis_' + i] = (train['primary_diagnosis'].str.contains(i) | 
                                    train['secondary_diagnosis'].str.contains(i) | 
                                    train['additional_diagnosis'].str.contains(i)).astype(int)
    
    # Drop original diagnosis columns from train
    train.drop(['primary_diagnosis', 'secondary_diagnosis', 'additional_diagnosis'], axis=1, inplace=True)
    
    # Step 3: Create binary columns for each unique diagnosis value in X_test
    for i in diagnosis_values:
        X_test['diagnosis_' + i] = (X_test['primary_diagnosis'].str.contains(i) | 
                                    X_test['secondary_diagnosis'].str.contains(i) | 
                                    X_test['additional_diagnosis'].str.contains(i)).astype(int)
    
    # Drop original diagnosis columns from X_test
    X_test.drop(['primary_diagnosis', 'secondary_diagnosis', 'additional_diagnosis'], axis=1, inplace=True)
    
    return train, X_test


In [7]:
def map_categorical_to_numeric(train, X_test, target_binary=None):
    # Mapping for 'gender'
    train['gender'] = train['gender'].map({'Male': 0, 'Female': 1})
    X_test['gender'] = X_test['gender'].map({'Male': 0, 'Female': 1})
  
    # Mapping for 'change_in_meds_during_hospitalization'
    train['change_in_meds_during_hospitalization'] = train['change_in_meds_during_hospitalization'].map({'No': 0, 'Ch': 1})
    X_test['change_in_meds_during_hospitalization'] = X_test['change_in_meds_during_hospitalization'].map({'No': 0, 'Ch': 1})
    
    # Mapping for 'prescribed_diabetes_meds'
    train['prescribed_diabetes_meds'] = train['prescribed_diabetes_meds'].map({'No': 0, 'Yes': 1})
    X_test['prescribed_diabetes_meds'] = X_test['prescribed_diabetes_meds'].map({'No': 0, 'Yes': 1})
    
    # Mapping for target_binary (if provided)
    if target_binary is not None:
        target_binary = target_binary.map({'No': 0, 'Yes': 1})
   
    return train, X_test, target_binary

In [8]:
# def map_categorical_to_numeric(train, X_test, target_binary):
#     # Mapping for 'gender'
#     train['gender'] = train['gender'].map({'Male': 0, 'Female': 1})
#     X_test['gender'] = X_test['gender'].map({'Male': 0, 'Female': 1})
    
#     # Mapping for 'change_in_meds_during_hospitalization'
#     train['change_in_meds_during_hospitalization'] = train['change_in_meds_during_hospitalization'].map({'No': 0, 'Ch': 1})
#     X_test['change_in_meds_during_hospitalization'] = X_test['change_in_meds_during_hospitalization'].map({'No': 0, 'Ch': 1})
    
#     # Mapping for 'prescribed_diabetes_meds'
#     train['prescribed_diabetes_meds'] = train['prescribed_diabetes_meds'].map({'No': 0, 'Yes': 1})
#     X_test['prescribed_diabetes_meds'] = X_test['prescribed_diabetes_meds'].map({'No': 0, 'Yes': 1})
    
#     # Mapping for target_binary
#     target_binary = target_binary.map({'No': 0, 'Yes': 1})
    
#     return train, X_test, target_binary

In [9]:
def label_encode_admission_type(train, X_test):
    # Define the mapping based on the specified order
    admission_type_mapping = {'Emergency': 1, 'Urgent': 2, 'Elective': 3, 'Newborn': 4, 'Trauma Center': 5}
    
    # Apply the mapping to 'admission_type' column in train and X_test
    train['admission_type'] = train['admission_type'].map(admission_type_mapping)
    X_test['admission_type'] = X_test['admission_type'].map(admission_type_mapping)
    
    return train, X_test

In [10]:
def map_admission_source(train, X_test):
    # Define the mapping function for 'admission_source' column
    map_func = lambda x: 'Transfer from Another Health Facility' if 'Transfer' in x else x
    
    # Apply the mapping function to 'admission_source' column in train and X_test
    train['admission_source'] = train['admission_source'].map(map_func)
    X_test['admission_source'] = X_test['admission_source'].map(map_func)
    
    return train, X_test

In [11]:
def one_hot_encode_and_combine(train, X_test):
    # Step 1: Identify categorical columns
    obj_cols = train.select_dtypes(include=['object']).columns.tolist()
    
    # Step 2: Initialize OneHotEncoder
    encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    
    # Step 3: Fit and transform on train
    X_train_encoded_bm = encoder.fit_transform(train[obj_cols])
    
    # Step 4: Transform X_test
    X_test_encoded_bm = encoder.transform(X_test[obj_cols])
    
    # Step 5: Get feature names after encoding
    encoded_columns = encoder.get_feature_names_out(obj_cols)
    
    # Step 6: Create DataFrames for encoded features
    X_train_encoded_bm_df = pd.DataFrame(X_train_encoded_bm, columns=encoded_columns, index=train.index)
    X_test_encoded_bm_df = pd.DataFrame(X_test_encoded_bm, columns=encoded_columns, index=X_test.index)
    
    # Step 7: Drop original categorical columns from train and X_test
    X_train_num = train.drop(obj_cols, axis=1)
    X_test_num = X_test.drop(obj_cols, axis=1)
    
    # Concatenate numeric and encoded categorical DataFrames
    train = pd.concat([X_train_num, X_train_encoded_bm_df], axis=1)
    X_test = pd.concat([X_test_num, X_test_encoded_bm_df], axis=1)
    
    return train, X_test

In [12]:
def drop_patient_id_column(train, X_test):
    # Drop 'patient_id' column from train and X_test
    train.drop('patient_id', axis=1, inplace=True)
    X_test.drop('patient_id', axis=1, inplace=True)
    
    return train, X_test

In [13]:
def scale_numerical_columns(train, X_test):
    # Step 1: Identify numerical columns
    numerical = train.select_dtypes(include=np.number).columns.tolist()
    
    # Step 2: Count unique values for each numerical column
    cat = {col: train[col].nunique() for col in numerical}
    
    # Step 3: Categorize columns into non-binary and binary
    non_binary = [col for col in cat if cat[col] != 2]
    binary = [col for col in cat if cat[col] == 2]
    
    # Step 4: Initialize RobustScaler
    scaler = RobustScaler()
    
    # Step 5: Fit and transform non-binary columns on train, and transform on X_test
    train.loc[:, non_binary] = scaler.fit_transform(train[non_binary])
    X_test.loc[:, non_binary] = scaler.transform(X_test[non_binary])
    
    return train, X_test

In [14]:
def select_features_with_models(train, X_test, target_binary):
    # Step 1: Random Forest Feature Importance
    rf_model = RandomForestClassifier(class_weight='balanced', random_state=42, max_depth=6, n_jobs=-1)
    rf_model.fit(train, target_binary)
    
    feature_importances = rf_model.feature_importances_
    feature_importance_df = pd.DataFrame({'Feature': train.columns, 'Importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
    sorted_indices = np.argsort(feature_importances)[::-1]
    top_30_features = sorted_indices[:30]
    selected_features = train.columns[top_30_features]
    
    selected = {}
    for i in selected_features:
        selected[i] = 1
    
    # Step 2: RFE with Random Forest took more than 10 minutes to run and it always returned the following 2 features
    selected_features = ['inpatient_visits_in_previous_year', 'hospital_visits']
    for i in selected_features:
        if i in selected.keys():
            selected[i] += 1
        else:
            selected[i] = 1
    
    # Step 3: Lasso Feature Selection
    reg = LassoCV().fit(train, target_binary)
    coef = pd.Series(reg.coef_, index=train.columns)
    print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
    
    lasso_features = coef[coef != 0].index.tolist()
    for i in lasso_features:
        if i in selected.keys():
            selected[i] += 1
        else:
            selected[i] = 1
    
    # Step 4: Select features to keep
    keep = [i for i in selected.keys() if selected[i] >= 2]
    
    # Step 5: Subset train and X_test DataFrames
    final_train = train[keep]
    final_X_test = X_test[keep]
    
    return final_train, final_X_test

In [15]:
 def all_preprocess(df, X_test):
     # Step 1: Prepare data
     train, target_binary = prepare_data(df)
   
     # Step 2: Process data
     train, X_test = process_data(train, X_test)
    
     # Step 3: Clean data
     train = clean_data(train)
     X_test = clean_data(X_test)
    
     # Step 4: Process age column
     train = process_age_column(train)
     X_test = process_age_column(X_test)
    
     # Step 5: Drop newborn outliers
     train, target_binary = drop_newborn_outliers(train, target_binary)
     X_test = drop_newborn_outliers1(X_test)
    
     # Step 6: Fill missing age
     train, X_test = fill_missing_age(train, X_test)
    
     # Step 7: Fill missing race
     train, X_test = fill_missing_race(train, X_test)
    
     # Step 8: Impute missing values
     train, X_test = impute_missing_values(train, X_test)
    
     # Step 9: Map ICD-9 to category
     train = map_icd9_to_category(train)
     X_test = map_icd9_to_category(X_test)
    
     # Step 10: Create diagnosis columns
     train, X_test = create_diagnosis_columns(train, X_test)
    
     # Step 11: Map categorical to numeric
     train, X_test, target_binary = map_categorical_to_numeric(train, X_test, target_binary)
    
     # Step 12: Label encode admission type
     train, X_test = label_encode_admission_type(train, X_test)
    
     # Step 13: Map admission source
     train, X_test = map_admission_source(train, X_test)
    
     # Step 14: One-hot encode and combine
     train, X_test = one_hot_encode_and_combine(train, X_test)
    
     # Step 15: Drop patient ID column
     train, X_test = drop_patient_id_column(train, X_test)
    
     # Step 16: Scale numerical columns
     train, X_test = scale_numerical_columns(train, X_test)

     train_preprocessed = train
     X_test_preprocessed = X_test

     return train_preprocessed, X_test_preprocessed, target_binary


In [16]:
train, X_test, target_binary = all_preprocess(df, X_test)

c:\Users\Estevao\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:972: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\Estevao\AppData\Local\Temp\ipykernel_30040\1348334179.py:16: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  train.loc[:, non_binary] = scaler.fit_transform(train[non_binary])
C:\Users\Estevao\AppData\Local\Temp\ipykernel_30040\1348334179.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, 

In [17]:
def save_data(df: pd.DataFrame, output_path: str):
    df.to_csv(output_path, index=False)
    return df

In [18]:
save_data(train, 'data/train_preprocessed.csv')

gender  age  payer_code  outpatient_visits_in_previous_year  \
encounter_id                                                                
533253             1  0.5         0.0                                 0.0   
426224             0 -0.5         0.0                                 0.0   
634063             1  0.0         0.0                                 0.0   
890610             0  0.0         1.0                                 0.0   
654194             1  0.5         1.0                                 1.0   
...              ...  ...         ...                                 ...   
660160             1  1.0         1.0                                 0.0   
826429             1  0.5         0.0                                 0.0   
332030             1  0.0         0.0                                 0.0   
757560             0  0.0         1.0                                 0.0   
459757             0  0.5         1.0                                 0.0   

              emergency_visits_in_previous_year  \
encounter_id                                      
533253                                      0.0   
426224                                      0.0   
634063                                      0.0   
890610                                      0.0   
654194                                      0.0   
...                                         ...   
660160                                      1.0   
826429                                      0.0   
332030                                      2.0   
757560                                      0.0   
459757                                      0.0   

              inpatient_visits_in_previous_year  admission_type  \
encounter_id                                                      
533253                                      2.0             0.0   
426224                                      0.0             0.0   
634063                                      1.0             0.0   
890610                                      1.0             0.0   
654194                                      0.0             2.0   
...                                         ...             ...   
660160                                      0.0             2.0   
826429                                      0.0             1.0   
332030                                      2.0             0.0   
757560                                      2.0             0.0   
459757                                      1.0             1.0   

              average_pulse_bpm  length_of_stay_in_hospital  number_lab_tests  \
encounter_id                                                                    
533253                 0.692308                       -0.50          0.153846   
426224                 0.717949                        2.50          1.038462   
634063                -0.153846                        0.50          0.615385   
890610                -0.487179                        0.50          0.153846   
654194                 0.666667                        0.50          0.115385   
...                         ...                         ...               ...   
660160                -0.256410                        0.75          0.269231   
826429                -0.051282                        1.25         -0.923077   
332030                 0.948718                        0.25         -0.730769   
757560                 0.589744                       -0.50         -0.384615   
459757                -0.974359                        0.25         -0.076923   

              ...  medication_['rosiglitazone', 'glyburide-metformin']  \
encounter_id  ...                                                        
533253        ...                                                0.0     
426224        ...                                                0.0     
634063        ...                                                0.0     
890610        ...                                   

In [18]:
#final_train.shape, final_X_test.shape 

In [19]:
# ((71231, 27), (30526, 27))

In [ ]:
def select_features_with_models(train, X_test, target_binary):
    # Step 1: Random Forest Feature Importance
    rf_model = RandomForestClassifier(class_weight='balanced', random_state=42, max_depth=6, n_jobs=-1)
    rf_model.fit(train, target_binary)
    
    feature_importances = rf_model.feature_importances_
    feature_importance_df = pd.DataFrame({'Feature': train.columns, 'Importance': feature_importances})
    feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
    sorted_indices = np.argsort(feature_importances)[::-1]
    top_30_features = sorted_indices[:30]
    selected_features = train.columns[top_30_features]
    
    selected = {}
    for i in selected_features:
        selected[i] = 1
    
    # Step 2: RFE with Random Forest took more than 10 minutes to run and it always returned the following 2 features
    selected_features = ['inpatient_visits_in_previous_year', 'hospital_visits']
    for i in selected_features:
        if i in selected.keys():
            selected[i] += 1
        else:
            selected[i] = 1
    
    # Step 3: Lasso Feature Selection
    reg = LassoCV().fit(train, target_binary)
    coef = pd.Series(reg.coef_, index=train.columns)
    print("Lasso picked " + str(sum(coef != 0)) + " variables and eliminated the other " +  str(sum(coef == 0)) + " variables")
    
    lasso_features = coef[coef != 0].index.tolist()
    for i in lasso_features:
        if i in selected.keys():
            selected[i] += 1
        else:
            selected[i] = 1
    
    # Step 4: Select features to keep
    keep = [i for i in selected.keys() if selected[i] >= 2]
    
    # Step 5: Subset train and X_test DataFrames
    final_train = train[keep]
    final_X_test = X_test[keep]
    
    return final_train, final_X_test

In [20]:
final_train, final_X_test = select_features_with_models(train, X_test, target_binary)
final_train.shape, final_X_test.shape 

Lasso picked 61 variables and eliminated the other 397 variables


((71231, 27), (30526, 27))

In [20]:
# train, target_multiclass, target_binary = prepare_data(df)

# train, X_test = process_data(train, X_test)

# train = clean_data(train)
# X_test = clean_data(X_test)

# train = process_age_column(train)
# X_test = process_age_column(X_test)

# train, target_binary = drop_newborn_outliers(train, target_binary)
# X_test = drop_newborn_outliers1(X_test)

# train, X_test = fill_missing_age(train, X_test)

# train, X_test = fill_missing_race(train, X_test)

# train, X_test = impute_missing_values(train, X_test)

# train = map_icd9_to_category(train)
# X_test = map_icd9_to_category(X_test)

# train, X_test = create_diagnosis_columns(train, X_test)

# train, X_test,target_binary = map_categorical_to_numeric(train, X_test, target_binary)

# train, X_test = label_encode_admission_type(train, X_test)

# train, X_test = map_admission_source(train, X_test)

# train, X_test = one_hot_encode_and_combine(train, X_test)

# train, X_test = drop_patient_id_column(train, X_test)

# train, X_test = scale_numerical_columns(train, X_test)

# final_train, final_X_test = select_features_with_models(train, X_test, target_binary)

# Checking for DataDrift

### 1st With Statistical Tests: Kolmogorov-Smirnov Test

In [21]:
from scipy.stats import ks_2samp

# Perform KS Test
ks_test_results = {}

for column in final_train.columns:
    ks_stat, p_value = ks_2samp(final_train[column], final_X_test[column])
    ks_test_results[column] = {'ks_stat': ks_stat, 'p_value': p_value}

# Print Results
print("KS Test Results between final_train and final_X_test:")
for feature, result in ks_test_results.items():
    print(f"{feature}: KS Statistic = {result['ks_stat']:.4f}, p-value = {result['p_value']:.4f}")

KS Test Results between final_train and final_X_test:
hospital_visits: KS Statistic = 0.1521, p-value = 0.0000
inpatient_visits_in_previous_year: KS Statistic = 0.0052, p-value = 0.6195
discharge_disposition_Discharged to home: KS Statistic = 0.0040, p-value = 0.8864
Emergency_visits/total_visits: KS Statistic = 0.0032, p-value = 0.9791
discharge_disposition_Discharged/transferred to another rehab fac including rehab units of a hospital .: KS Statistic = 0.0001, p-value = 1.0000
discharge_disposition_Expired: KS Statistic = 0.0007, p-value = 1.0000
number_of_medications: KS Statistic = 0.0065, p-value = 0.3181
number_diagnoses: KS Statistic = 0.0027, p-value = 0.9973
length_of_stay_in_hospital: KS Statistic = 0.0062, p-value = 0.3771
discharge_disposition_Discharged/transferred to SNF: KS Statistic = 0.0006, p-value = 1.0000
medication_['insulin']: KS Statistic = 0.0019, p-value = 1.0000
age: KS Statistic = 0.0063, p-value = 0.3594
outpatient_visits_in_previous_year: KS Statistic = 0.0

Significant Drift: Features with high KS statistic and low p-value indicate significant data drift. These features might need closer attention as they could affect model performance if not handled properly.

No Significant Drift: Features with low KS statistic and/or high p-value suggest that the distributions are similar between final_train and final_X_test, indicating less concern for data drift in these features.

#### Only the feature "hospital_visits" showed significant data drift, as indicated by a high KS statistic and a very low p-value (0.0000).

### Machine Learning-Based Approaches: Adversarial Validation

In [22]:
# Add target column
final_train['is_train'] = 1
final_X_test['is_train'] = 0

# Concatenate final_train and final_X_test
combined_data = pd.concat([final_train, final_X_test], axis=0)

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

# Separate features and target
X = combined_data.drop(['is_train'], axis=1)
y = combined_data['is_train']

# Split into train and test sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a classifier
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Predict probabilities
y_pred_proba = clf.predict_proba(X_val)[:, 1]

# Evaluate model performance
auc_score = roc_auc_score(y_val, y_pred_proba)
print(f"AUC Score (Adversarial Validation): {auc_score}")

AUC Score (Adversarial Validation): 0.5895356367951301


#### An AUC score of 0.588 in Adversarial Validation indicates some level of distinguishability between the training and test datasets, suggesting potential data drift or differences that could impact model performance.

### Distance Metrics: Jensen-Shannon Divergence

In [23]:
import numpy as np
from scipy.stats import entropy
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KernelDensity

def calculate_distribution(data):
    # Scale data
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(data)
    
    # Fit kernel density estimator
    kde = KernelDensity(bandwidth=0.5, kernel='gaussian')
    kde.fit(scaled_data)
    
    # Calculate log-probability density
    log_density = kde.score_samples(scaled_data)
    density = np.exp(log_density)
    
    return density

def jensen_shannon_divergence(p, q):
    # Compute JSD
    m = 0.5 * (p + q)
    jsd = 0.5 * (entropy(p, m) + entropy(q, m))
    return jsd

def calculate_jsd_for_column(data1, data2, col):
    # Calculate distributions for the specified column
    dist1 = calculate_distribution(data1[[col]])
    dist2 = calculate_distribution(data2[[col]])
    
    # Ensure distributions are of the same length
    min_length = min(len(dist1), len(dist2))
    dist1 = dist1[:min_length]
    dist2 = dist2[:min_length]
    
    # Calculate Jensen-Shannon Divergence
    jsd = jensen_shannon_divergence(dist1, dist2)
    
    return jsd

# # Example usage for a single column
# column_name = 'hospital_visits'
# jsd_score = calculate_jsd_for_column(final_train, final_X_test, column_name)

# print(f"Jensen-Shannon Divergence score for '{column_name}': {jsd_score}")

Jensen-Shannon Divergence score for 'hospital_visits': 0.06434938653854827

This low score indicates that there is minimal data drift for the 'hospital_visits' feature, meaning that the distributions of this feature in the training and test sets are quite similar.

### Features with possible drift according to the KS: number_of_medications, length_of_stay_in_hospital, n_medications/length_of_stay, age

In [24]:
# column_name = 'number_of_medications'
# jsd_score = calculate_jsd_for_column(final_train, final_X_test, column_name)

# print(f"Jensen-Shannon Divergence score for '{column_name}': {jsd_score}")

#### Jensen-Shannon Divergence score for 'number_of_medications': 0.046017138014232274, Same Conclusion

In [25]:
# column_name = 'length_of_stay_in_hospital'
# jsd_score = calculate_jsd_for_column(final_train, final_X_test, column_name)

# print(f"Jensen-Shannon Divergence score for '{column_name}': {jsd_score}")

Jensen-Shannon Divergence score for 'length_of_stay_in_hospital': 0.04820765880788821, Same Conclusion

In [26]:
# column_name = 'n_medications/length_of_stay'
# jsd_score = calculate_jsd_for_column(final_train, final_X_test, column_name)

# print(f"Jensen-Shannon Divergence score for '{column_name}': {jsd_score}")

Jensen-Shannon Divergence score for 'n_medications/length_of_stay': 0.05841328258906302, Same Conclusion

In [27]:
# column_name = 'age'
# jsd_score = calculate_jsd_for_column(final_train, final_X_test, column_name)

# print(f"Jensen-Shannon Divergence score for '{column_name}': {jsd_score}")

Jensen-Shannon Divergence score for 'age': 0.039089801272174506, same conclusion

### Overall, the diferent methods to check for Data Drift, demonstrated that there is some but its small

### The Kolmogorov-Smirnov Test, only indicated data drift in one feature -> 'hospital_visits'

### The Adversarial Validation showed some distinguishability, suggesting potential for data drift

### And the Jensen-Shannon Divergence showed minimal drift across the dataset

MUltivariate extra


In [28]:
#pip install evidently

In [29]:
#pip install nannyml

In [30]:
#pip install utils

In [31]:
# import logging
# from typing import Any, Dict, Tuple

# import numpy as np
# import pandas as pd

# from great_expectations.core import ExpectationSuite, ExpectationConfiguration
# import great_expectations as gx

# from pathlib import Path

# from kedro.config import OmegaConfigLoader
# from kedro.framework.project import settings

# #from utils import calculate_psi
# import matplotlib.pyplot as plt  
# import nannyml as nml
# from evidently.report import Report
# from evidently.metric_preset import DataDriftPreset

In [32]:
# import pandas as pd
# import nml
# from evidently.dashboard import Dashboard
# from evidently.tabs import DataDriftTab

def data_drift(data_reference: pd.DataFrame, data_analysis: pd.DataFrame):
    # Define the threshold for the test as parameters in the parameters catalog
    constant_threshold = nml.thresholds.ConstantThreshold(lower=None, upper=0.2)
    constant_threshold.thresholds(data_reference)

    # Initialize the object that will perform the Univariate Drift calculations
    univariate_calculator = nml.UnivariateDriftCalculator(
        column_names=list(data_reference.columns),
        treat_as_categorical=list(data_reference.select_dtypes(include=['object', 'category']).columns),
        chunk_size=50,
        categorical_methods=['jensen_shannon'],
        thresholds={"jensen_shannon": constant_threshold}
    )

    # Fit the drift calculator with reference data
    univariate_calculator.fit(data_reference)
     # Calculate drift for analysis data and filter results for visualization
    results = univariate_calculator.calculate(data_analysis).filter(period='analysis').to_df()
     # Plot drift for visualization and save as HTML
    figure = univariate_calculator.calculate(data_analysis).plot(methods=['jensen_shannon'], kind='drift')
    figure.write_html("univariate_nml.html")

    # Generate a report using KS test for numeric features
    data_drift_report = Dashboard(tabs=[DataDriftTab])
    data_drift_report.calculate(current_data=data_analysis, reference_data=data_reference)
    data_drift_report.save("data_drift_report.html")

    return results

# Assuming final_train and final_X_test are defined DataFrames
# Call data_drift function with your datasets
results = data_drift(final_train, final_X_test)